In [ ]:
import pandas as pd
import os
import glob

In [ ]:
data_path = '/Users/sonynka/HTW/IC/data/fashionid/'

In [ ]:
df = pd.read_csv(os.path.join(data_path, 'data.csv'), sep=';')

In [ ]:
df.head()

In [ ]:
print('Data points', df.shape[0])
print(df.groupby('category')['img_path'].count())

# Clean data
## Remove Duplicates

Drop duplicated rows which could occured during scraping data and appending to csvs.

If the count of img_path is more than unique, drop the duplicated img_path. This can occurr, if the same image is assigned to two different colors. In that case, just keep the first color.

*Note: We apply this to img_path, since it is acceptable that img_url is duplicated -> one image can be assigned to two different subcategories, such as Kurze Kleider und Cocktailkleider. If one image should only have one subcategory assigned, duplicated should be dropped on img_url not on img_path.*

In [ ]:
print('Shape', df.shape)
print('Without duplicates', df.drop_duplicates().shape)

## Check images and CSVs consistency
### CSV -> image path check
Make sure all the image paths in the csvs actually exist in the folder structure.

In [ ]:
df['exists'] = [True if os.path.exists(os.path.join(data_path, str(x))) else False for x in df['img_path']]
not_exist_imgs_count = df[df['exists'] == False].shape[0]
print('Not existing images', not_exist_imgs_count)
print(df[df['exists'] == False])

In [ ]:
df = df[df['exists'] == True]

In [ ]:
df = df.drop('exists', axis=1)

### image path -> CSV check
Make sure all the images in the folders have an entry in the csv.

In [ ]:
for category in os.listdir(data_path):
    
    category_path = os.path.join(data_path, category)
    if os.path.isdir(category_path):
        print(category)
        img_names = [os.path.join(category, o) for o in os.listdir(category_path)]
        row_names = df[df['category'] == category]['img_path']
        diff_imgs = list(set(img_names).difference(row_names))
        print('Diff', len(diff_imgs))
        
        print(diff_imgs)
    

In [ ]:
df.to_csv(os.path.join(data_path, 'data.csv'), sep=';', encoding='utf-8', index=False)